# API Keys

In [ ]:
from dotenv import load_dotenv

load_dotenv()

# Messages

In [ ]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

messages = [AIMessage(content="So you said you were researching ocean mammals?", name="Model")]
messages.extend([HumanMessage(content="Yes, that's right.", name="Lance")])
messages.extend([AIMessage(content="Great, what would you like to learn about.", name="Model")])
messages.extend([HumanMessage(content="I want to learn about the best place to see Orcas in the Us.", name="Lance")])

for message in messages:
    message.pretty_print()

# Chat Models

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

result = llm.invoke(messages)

In [ ]:
type(result)

In [ ]:
result

# Tools

In [ ]:
def multiply(a: int, b: int) -> int:
    """
    Multiply a and b.

    Args:
        a: first int
        b: second int
    """

    return a * b

llm_with_tools = llm.bind_tools([multiply])

In [ ]:
tool_call = llm_with_tools.invoke([HumanMessage(content="What is 2 multipled by 3?", name="Tiago")])

In [ ]:
tool_call

In [ ]:
tool_call.additional_kwargs['function_call']

# Using messages as state

In [ ]:
from typing import TypedDict
from langchain_core.messages import AnyMessage

class MessageState(TypedDict):
    messages: list[AnyMessage]

# Reducers

In [ ]:
from typing import Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

Como essa é uma estrutura muito comum, LangGraph possui um MessagesState pronto para uso:

In [ ]:
from langgraph.graph import MessagesState

class State(MessagesState):
    pass

In [ ]:
# Estado inicial
initial_messages = [
    AIMessage(content="Hello ! How can I assist you?", name="Model"),
    HumanMessage(content="I'm looking for information on marine biology.", name="Tiago")
]

# Nova menssagem para adicionar
new_message = AIMessage(content="Sure, I can help with that. What specifically are you interested in?", name="Model")

# Testando
add_messages(initial_messages, new_message)

# Our Graph

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# State
class MessagesState(MessagesState):
    pass

# Node
def tool_calling_llm(state: MessagesState):
    new_message = llm_with_tools.invoke(state["messages"])
    return {"messages": [new_message]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_edge(START, "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Hello!")})
messages

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Multiply 2 and 3.")})
messages